In [1]:
import pandas as pd
import numpy as np

In [2]:
# submit file from @antklen (precision ~ 0.32, R1 ~ 0.0831)
S1 = pd.read_csv('../submit_data/subm_0-0831.csv')

# submit files from (v5_daily_cross.ipynb, low precision, high mrr)
S2 = pd.read_csv('../submit_data/cross_v1_7_200.csv')
S3 = pd.read_csv('../submit_data/cross_v1.csv')
S4 = pd.read_csv('../submit_data/cross_v3.csv')

In [3]:
def listify(x):
    rtks = x.split(', ')
    rtks[0] = rtks[0][1:]
    rtks[-1] = rtks[-1][:-1]
    return rtks

In [4]:
S1['rtk'] = S1['rtk_list'].apply(listify)
S2['rtk'] = S2['rtk_list'].apply(listify)
S3['rtk'] = S3['rtk_list'].apply(listify)
S4['rtk'] = S4['rtk_list'].apply(listify)

In [5]:
del S1['rtk_list']
del S2['rtk_list']
del S3['rtk_list']
del S4['rtk_list']

In [6]:
S1 = S1.explode('rtk')
S2 = S2.explode('rtk')
S3 = S3.explode('rtk')
S4 = S4.explode('rtk')

In [7]:
S1.head()

,bank,rtk
0,001708a5fca04c2ca58d7188b747d5f5,6a3f50c8ab3246e2a240cb98475c383f
0,001708a5fca04c2ca58d7188b747d5f5,9ca9be97d4cd4d278cb8425aad3dfc97
0,001708a5fca04c2ca58d7188b747d5f5,81facf32e0504ad190e002b22318f48d
0,001708a5fca04c2ca58d7188b747d5f5,1fca01621c3440c39003b05aa2b10be2
0,001708a5fca04c2ca58d7188b747d5f5,117c8ebe50f240949a1f24004786bb02


In [8]:
S1['rank'] = S1.groupby('bank').cumcount()
S2['rank2'] = S2.groupby('bank').cumcount()
S3['rank3'] = S3.groupby('bank').cumcount()
S4['rank4'] = S4.groupby('bank').cumcount()

In [9]:
S = pd.merge(S1, S2, how='left', on=['bank', 'rtk'])
S = pd.merge(S, S3, how='left', on=['bank', 'rtk'])
S = pd.merge(S, S4, how='left', on=['bank', 'rtk'])

In [10]:
S.head()

,bank,rtk,rank,rank2,rank3,rank4
0,001708a5fca04c2ca58d7188b747d5f5,6a3f50c8ab3246e2a240cb98475c383f,0,NaN,NaN,NaN
1,001708a5fca04c2ca58d7188b747d5f5,9ca9be97d4cd4d278cb8425aad3dfc97,1,NaN,NaN,NaN
2,001708a5fca04c2ca58d7188b747d5f5,81facf32e0504ad190e002b22318f48d,2,NaN,NaN,NaN
3,001708a5fca04c2ca58d7188b747d5f5,1fca01621c3440c39003b05aa2b10be2,3,NaN,NaN,NaN
4,001708a5fca04c2ca58d7188b747d5f5,117c8ebe50f240949a1f24004786bb02,4,NaN,NaN,NaN


In [11]:
S = S.sort_values(['bank', 'rank2', 'rank3', 'rank4', 'rank'])

In [12]:
# доля записей, которые пересортировались
S[(~S['rank2'].isnull()) | (~S['rank3'].isnull()) | (~S['rank4'].isnull())].shape[0]/S.shape[0]

0.07975969305331179

In [13]:
S = S[['bank', 'rtk']]
S = S.pivot_table(index='bank', values='rtk', aggfunc=list)
S = S.reset_index()
S = S[['bank', 'rtk']]
S.columns = ['bank', 'rtk_list']

In [14]:
S.shape

(4952, 2)

In [15]:
S.to_csv('../submit_data/hybrid_submit.csv', index=None)

In [16]:
S.head()

,bank,rtk_list
0,001708a5fca04c2ca58d7188b747d5f5,"[02831d884f5445c0a6acd600752e8630, 6a3f50c8ab3..."
1,0025930588d249ec8e73936f6a6d05bd,"[a477046586084eadab508b996dcb273c, 9b459f90d97..."
2,0034020d25da4951b559da1d3171a1e3,"[22dd90b05611479cb8ab7d29092a3aaf, c6216c3cbe2..."
3,00491339a09c4638a0f27ca51f615d67,"[cc0070c7f5484076a69f4465addfab26, 010e097f57b..."
4,006415c4240844e0810617ecebbd2db4,"[c5e93edb6f914acf83b2a3e0bec43b66, 3b323c590d3..."
